5220411122
MUHAMMAD HASBY MUHARRAMAN

In [1]:
!pip install pyspark
#!pip install pyspark.SparkSession


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

24/01/12 15:40:39 WARN Utils: Your hostname, codespaces-f75b5f resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/01/12 15:40:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/12 15:40:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/12 15:40:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = spark.read.csv('Car_Purchasing_Data.csv', header=True, inferSchema=True)
data.show()

+---------------+--------------------+-------+------+---+-------------+----------------+-----------+-------------------+
|  Customer Name|     Customer e-mail|Country|Gender|Age|Annual Salary|Credit Card Debt|  Net Worth|Car Purchase Amount|
+---------------+--------------------+-------+------+---+-------------+----------------+-----------+-------------------+
|  Martina Avila|cubilia.Curae.Pha...|    USA|     0| 42|  62812.09301|     11609.38091|238961.2505|        35321.45877|
|  Harlan Barnes| eu.dolor@diam.co.uk|    USA|     0| 41|  66646.89292|     9572.957136|530973.9078|        45115.52566|
|Naomi Rodriquez|vulputate.mauris....|    USA|     1| 43|  53798.55112|     11160.35506|638467.1773|        42925.70921|
|Jade Cunningham|malesuada@digniss...|    USA|     1| 58|  79370.03798|     14426.16485|548599.0524|        67422.36313|
|   Cedric Leach|felis.ullamcorper...|    USA|     1| 57|   59729.1513|     5358.712177|560304.0671|        55915.46248|
|   Carla Hester|  mi@Aliquamera

In [5]:
data.printSchema()

root
 |-- Customer Name: string (nullable = true)
 |-- Customer e-mail: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Salary: double (nullable = true)
 |-- Credit Card Debt: double (nullable = true)
 |-- Net Worth: double (nullable = true)
 |-- Car Purchase Amount: double (nullable = true)



In [6]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Customer Name',
 'Customer e-mail',
 'Country',
 'Gender',
 'Age',
 'Annual Salary',
 'Credit Card Debt',
 'Net Worth',
 'Car Purchase Amount']

In [8]:
assembler = VectorAssembler(
    inputCols = ['Annual Salary', 'Credit Card Debt', 'Net Worth', 'Car Purchase Amount'],
    outputCol = 'features'
)

In [9]:
output = assembler.transform(data)

In [10]:
output.select("features").show(4)

+--------------------+
|            features|
+--------------------+
|[62812.09301,1160...|
|[66646.89292,9572...|
|[53798.55112,1116...|
|[79370.03798,1442...|
+--------------------+
only showing top 4 rows



In [11]:
output.show(1)

+-------------+--------------------+-------+------+---+-------------+----------------+-----------+-------------------+--------------------+
|Customer Name|     Customer e-mail|Country|Gender|Age|Annual Salary|Credit Card Debt|  Net Worth|Car Purchase Amount|            features|
+-------------+--------------------+-------+------+---+-------------+----------------+-----------+-------------------+--------------------+
|Martina Avila|cubilia.Curae.Pha...|    USA|     0| 42|  62812.09301|     11609.38091|238961.2505|        35321.45877|[62812.09301,1160...|
+-------------+--------------------+-------+------+---+-------------+----------------+-----------+-------------------+--------------------+
only showing top 1 row



In [12]:
final_data = output.select('features', 'Annual Salary')
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Annual Salary: double (nullable = true)



In [13]:
final_data.show(3)

+--------------------+-------------+
|            features|Annual Salary|
+--------------------+-------------+
|[62812.09301,1160...|  62812.09301|
|[66646.89292,9572...|  66646.89292|
|[53798.55112,1116...|  53798.55112|
+--------------------+-------------+
only showing top 3 rows



In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
lr = LinearRegression(labelCol='Annual Salary')

In [16]:
lr_model = lr.fit(train_data)

24/01/12 15:40:50 WARN Instrumentation: [91587b19] regParam is zero, which might cause numerical instability and overfitting.
24/01/12 15:40:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/12 15:40:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/01/12 15:40:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [17]:
print('koefisien : {}\nintercept : {} '.format(lr_model.coefficients, lr_model.intercept))

koefisien : [0.9999999999999972,1.0365009846133808e-14,-3.621914321152136e-16,4.749644104033246e-15]
intercept : 3.044111479099381e-11 


In [18]:
test_result = lr_model.evaluate(test_data)

In [19]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|                 0.0|
|-3.63797880709171...|
|2.182787284255027...|
|-2.18278728425502...|
|-8.00355337560176...|
|-2.91038304567337...|
|3.637978807091713...|
|-4.36557456851005...|
|3.637978807091713...|
|3.637978807091713...|
|-1.45519152283668...|
|1.455191522836685...|
|3.637978807091713...|
|7.275957614183426...|
|9.458744898438454...|
|5.820766091346741...|
|-2.91038304567337...|
|-7.27595761418342...|
|-1.45519152283668...|
|-3.63797880709171...|
+--------------------+
only showing top 20 rows



In [20]:
# RMSE (Root, Mean, Square, Error)
print('RMSE : {}'.format(test_result.rootMeanSquaredError))

RMSE : 6.194314091194576e-11


In [21]:
# MSE (Root, Mean, Square, Error)
print('MSE : {}'.format(test_result.meanSquaredError))

MSE : 3.8369527060371685e-21


unlable data

In [22]:
unlable_data = test_data.select('features')

In [23]:
prediction = lr_model.transform(unlable_data)
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[33422.99683,8570...|       33422.99683|
|[34154.77654,5316...|34154.776540000035|
|[35069.41886,1851...|35069.418859999976|
|[37336.3383,10218...|37336.338300000025|
|[38453.86033,1528...| 38453.86033000008|
|[39460.00348,8769...| 39460.00348000003|
|[40558.75456,4981...|40558.754559999965|
|[41236.36497,9399...|41236.364970000046|
|[41587.39238,5397...| 41587.39237999996|
|[42415.48867,5205...| 42415.48866999996|
|[42433.54619,7335...|42433.546190000015|
|[42978.34284,8884...| 42978.34283999998|
|[43131.78411,1091...|43131.784109999964|
|[43412.86301,7843...|43412.863009999994|
|[44747.66132,4975...|44747.661319999905|
|[46549.16329,640....| 46549.16328999994|
|[47228.35999,9046...|47228.359990000026|
|[47460.54809,7405...|47460.548090000004|
|[48716.67271,1088...| 48716.67271000001|
|[49393.46784,1152...|49393.467840000034|
+--------------------+------------

In [24]:
# MSE
print('MSE : {} '.format(test_result.meanSquaredError))

# RMSE
print('RMSE : {} '.format(test_result.rootMeanSquaredError))

MSE : 3.8369527060371685e-21 
RMSE : 6.194314091194576e-11 


24/01/12 15:40:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [26]:
print('Selesai !')
print('Terima kasih untuk semua tugasnya dalam semester 3 ini pak.....\n')

Selesai !
Terima kasih untuk semua tugasnya dalam semester 3 ini pak.....

